In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier # using 1NN
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm 
from torchvision.utils import make_grid
from torchvision.utils import save_image
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Download the [cow teat datasets](https://github.com/YoushanZhang/SCTL) (10 points) resize image to (224, 224)

In [ ]:
def train_data_loader(train_dirs):
    # Initialize empty lists for to store image arrays and labels
    image_arrays = []
    image_labels = []
   
    # loop to navitage all directories and read information from all the image files
    for label, directory in enumerate(sorted(os.listdir(train_dirs))):
        path = os.path.join(train_dirs, directory)

        # check if it's a directory
        if not os.path.isdir(path):
            continue

        # Iterate over image files in the directory
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)

            # resize and store images
            img = cv2.resize(plt.imread(img_path).copy(), (224, 224))
            image_labels.append(label)
            image_arrays.append(img)
            
    return image_arrays, image_labels


def test_data_loader(data_dir):
    # Get a list of image files in the test data directory
    image_files = [f for f in os.listdir(data_dir) if f.endswith(".jpg")]
    
    # Initialize lists for data and file names
    image_arrays = []
    image_names = []
    
    # Iterate over the image files
    for file in image_files:
        # Get the full path of the image
        path = os.path.join(data_dir, file)
        
        # Read and preprocess the image
        img = cv2.resize(plt.imread(path).copy(), (224, 224))
        #img = img.astype(np.float32) / 255.0
        
        # Append the preprocessed image and file name to the lists
        image_arrays.append(img)
        image_names.append(file)
    
    # Convert lists to NumPy arrays and return
    return image_arrays, image_names

train_loader = r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Training'
training_array, training_labels = train_data_loader(train_loader)
training_array, training_labels = np.array(training_array), np.array(training_labels)

test_loader=r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Test_Data'
test_array, test_file_name = test_data_loader(test_loader)
test_array, test_file_name = np.array(test_array), np.array(test_file_name)


### (1). Create a train data loader that returns image arrays and labels

In [2]:
def train_data_loader(train_dirs):
    # Initialize empty lists for to store image arrays and labels
    image_arrays = []
    image_labels = []
   
    # loop to navitage all directories and read information from all the image files
    for label, directory in enumerate(sorted(os.listdir(train_dirs))):
        path = os.path.join(train_dirs, directory)

        # check if it's a directory
        if not os.path.isdir(path):
            continue

        # Iterate over image files in the directory
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)

            # resize and store images
            img = cv2.resize(plt.imread(img_path).copy(), (224, 224))
            image_labels.append(label)
            image_arrays.append(img)
            
    return image_arrays, image_labels

### (2). Create a test data loader that returns image arrays and file names

In [3]:
def test_data_loader(data_dir):
    # Get a list of image files in the test data directory
    image_files = [f for f in os.listdir(data_dir) if f.endswith(".jpg")]
    
    # Initialize lists for data and file names
    image_arrays = []
    image_names = []
    
    # Iterate over the image files
    for file in image_files:
        # Get the full path of the image
        path = os.path.join(data_dir, file)
        
        # Read and preprocess the image
        img = cv2.resize(plt.imread(path).copy(), (224, 224))
        #img = img.astype(np.float32) / 255.0
        
        # Append the preprocessed image and file name to the lists
        image_arrays.append(img)
        image_names.append(file)
    
    # Convert lists to NumPy arrays and return
    return image_arrays, image_names


### (3). Print image arrays, labels and file names dimensions 

### Training Dataset

In [4]:
train_loader = r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Training'
training_array, training_labels = train_data_loader(train_loader)
training_array, training_labels = np.array(training_array), np.array(training_labels)

print("Training Dataset Array:",training_array.shape )
print("Training Label Shape:",training_labels.shape )

Training Dataset Array: (1149, 224, 224, 3)
Training Label Shape: (1149,)


In [31]:
training_labels

array([0, 0, 0, ..., 3, 3, 3])

**Observation:**
   - The training dataset contains total 1149 images of size 224, 224 and has 3 channels.

### Test Dataset

In [5]:
# Example usage
test_loader=r'C:\AKA\Backup Dell Laptop\D Drive\YU\Semester 2\Neural Network\DLNN\Assignment_Week5\Homework Week5\Homework Week5\Test_Data'
test_array, test_file_name = test_data_loader(test_loader)
test_array, test_file_name = np.array(test_array), np.array(test_file_name)

print("Test Dataset Array:",test_array.shape )
print("Test File Names Extracted:",test_file_name.shape )


Test Dataset Array: (380, 224, 224, 3)
Test File Names Extracted: (380,)


**Observation:**
   - The test dataset contains total 380 images of size 224, 224 and has 3 channels.

In [32]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [33]:
# # load the dataset, split into input (X) and output (y) variables
# dataset = np.loadtxt('https://raw.githubusercontent.com/ankitagg2008/Misc/main/pima-indians-diabetes.csv', delimiter=',')
# X = dataset[:,0:8]
# y = dataset[:,8]

In [34]:
X = torch.tensor(training_array, dtype=torch.float32)
y = torch.tensor(training_labels, dtype=torch.float32).reshape(-1, 1)

In [38]:
X.shape

torch.Size([1149, 224, 224, 3])

In [39]:
model = nn.Sequential(
    nn.Linear(8, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid())

In [40]:
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)


In [41]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [42]:
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [43]:
n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (501760x3 and 8x12)

In [22]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)

accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.7747395634651184


In [23]:
# make probability predictions with the model
predictions = model(X)
# round predictions
rounded = predictions.round()

In [24]:
...
# make class predictions with the model
predictions = (model(X) > 0.5).int()

In [25]:
# compute accuracy
y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")
 
# make class predictions with the model
predictions = (model(X) > 0.5).int()
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

Accuracy 0.7747395634651184
[6.0, 148.0, 72.0, 35.0, 0.0, 33.599998474121094, 0.6269999742507935, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.600000381469727, 0.35100001096725464, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.299999237060547, 0.671999990940094, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.100000381469727, 0.16699999570846558, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.099998474121094, 2.2880001068115234, 33.0] => 1 (expected 1)


In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you have 'X_train', 'X_test', 'y_train', and 'y_test' for image data
# X_train and X_test should be numpy arrays containing image data
# y_train and y_test should be numpy arrays containing corresponding labels

# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(training_array, training_labels, test_size=0.2, random_state=42)

# Convert image data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

# Define the model for image data
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=224, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Adjust the input size based on your image dimensions
        self.fc1 = nn.Linear(16 * 224 * 224 // 4, 64)
        self.fc2 = nn.Linear(64, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 224 * 224 // 4)  # Adjust the division factor (// 4) to match the pooling size
        x = torch.relu(self.fc1(x))
        x = self.act_output(self.fc2(x))
        return x




# Train the model
loss_fn = nn.BCELoss()  # Binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10
batch_size = 16

# Create DataLoader for training and testing data
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

for epoch in range(n_epochs):
    for Xbatch, ybatch in train_loader:
        y_pred = model(Xbatch)
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluate the model on the test data
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()  # Set the model to evaluation mode
predictions = []

with torch.no_grad():
    for Xbatch, _ in test_loader:
        y_pred = model(Xbatch)
        predictions.append((y_pred > 0.5).int())

predictions = torch.cat(predictions)
accuracy = (predictions == y_test).float().mean()
print(f"Test Accuracy: {accuracy.item():.2f}")

# Make class predictions with the model (if needed)
for i in range(5):
    X_example = X_test[i].unsqueeze(0)
    prediction = (model(X_example) > 0.5).int()
    print(f"Prediction: {prediction.item()} (expected: {y_test[i].item()})")


RuntimeError: Given input size: (16x222x1). Calculated output size: (16x111x0). Output size is too small

In [54]:
X_train.shape

torch.Size([919, 224, 224, 3])

In [58]:
X_test.shape

torch.Size([230, 224, 224, 3])

In [60]:
y_train.shape

torch.Size([919, 1])

In [63]:
y_test.shape

torch.Size([230, 1])

In [64]:
pip install torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [71]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(64, 3)  # Replace 'num_classes' with the number of classes in your dataset

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.fc3(x)
        return x


In [72]:
import torch.optim as optim

# Define the loss function (e.g., CrossEntropy for classification)
criterion = nn.CrossEntropyLoss()

# Define the optimizer (e.g., Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [75]:
# Assuming you have loaded your training data into 'training_array' and 'training_labels'



# Convert data to PyTorch tensors
training_data = torch.Tensor(training_array)
training_labels = torch.LongTensor(training_labels)

# Define the number of classes (replace with the actual number)
num_classes = 10  # Change to your actual number of classes

# Initialize your neural network
model = NeuralNetwork()

# Set the device (CPU or GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)


# Assuming 'training_data' has shape [1149, 224, 224, 3]
# You can permute the dimensions to match the expected shape [1149, 3, 224, 224]
training_data = training_data.permute(0, 3, 1, 2)


# Training loop
num_epochs = 10  # You can adjust this as needed
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(training_data.to(device))
    loss = criterion(outputs, training_labels.to(device))
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the trained model
torch.save(model.state_dict(), 'trained_model.pth')


cuda:0


RuntimeError: CUDA out of memory. Tried to allocate 3.44 GiB (GPU 0; 8.00 GiB total capacity; 18.43 GiB already allocated; 0 bytes free; 20.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF